In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configure visualization styles
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Load Excel data (update file path as needed)
df = pd.read_csv(r'C:\Users\Kabelo\OneDrive\Documents\Power Learn Program\Pyt[final project]\owid-covid-data.csv', usecols=columns_to_use)


# Inspect the dataframe structure and columns
print("Columns in dataset:", df.columns.tolist())
print("\nBasic data info:")
print(df.info())

# Ensure required columns exist
required_cols = ['date', 'country', 'total_cases', 'total_deaths', 'new_cases']
missing_cols = [col for col in required_cols if col not in df.columns]

# Convert 'date' column to datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')
if df['date'].isnull().any():
    print("Warning: Some 'date' values could not be converted and are NaT.")

# Fill missing numeric data with 0 (you may choose other strategies)
for col in ['total_cases', 'total_deaths', 'new_cases']:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

# --- Descriptive statistics ---
print("\nDescriptive statistics for numeric columns:")
print(df[required_cols[2:]].describe())

# --- Task 1: Total cases over time for selected countries ---
selected_countries = ['USA', 'India', 'Brazil']  # customize this list
plt.figure()
for country in selected_countries:
    c_data = df[df['country'] == country].sort_values('date')
    plt.plot(c_data['date'], c_data['total_cases'], label=country)
plt.title('Total COVID-19 Cases Over Time')
plt.xlabel('Date')
plt.ylabel('Total Cases')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# --- Task 2: Total deaths over time (aggregated across all countries) ---
deaths_over_time = df.groupby('date')['total_deaths'].sum().reset_index()
plt.figure()
plt.plot(deaths_over_time['date'], deaths_over_time['total_deaths'], color='red')
plt.title('Total Deaths Over Time (All Countries)')
plt.xlabel('Date')
plt.ylabel('Total Deaths')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# --- Task 3: Compare daily new cases between selected countries ---
plt.figure()
for country in selected_countries:
    c_data = df[df['country'] == country].sort_values('date')
    plt.plot(c_data['date'], c_data['new_cases'], label=country)
plt.title('Daily New Cases Comparison')
plt.xlabel('Date')
plt.ylabel('Daily New Cases')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# --- Task 4: Calculate death rate (total_deaths / total_cases) ---
df['death_rate'] = 0
non_zero_cases = df['total_cases'] > 0
df.loc[non_zero_cases, 'death_rate'] = df.loc[non_zero_cases, 'total_deaths'] / df.loc[non_zero_cases, 'total_cases']

print("\nSample death rate values:")
print(df['country', 'date', 'death_rate'].head())

# --- Visualization: Bar chart of top 10 countries by total cases on latest date ---
latest_date = df['date'].max()
latest_data = df[df['date'] == latest_date]

if latest_data.empty:
    print("No data available for the latest date.")
else:
    top_countries = latest_data.sort_values('total_cases', ascending=False).head(10)
    plt.figure()
    sns.barplot(x='total_cases', y='country', data=top_countries, palette='magma')
    plt.title(f'Top 10 Countries by Total Cases on {latest_date.date()}')
    plt.xlabel('Total Cases')
    plt.ylabel('Country')
    plt.tight_layout()
    plt.show()

   
    numeric_cols = ['total_cases', 'total_deaths', 'new_cases', 'death_rate','country']
    corr_data = latest_data[numeric_cols].corr()

    plt.figure(figsize=(8,6))
    sns.heatmap(corr_data, annot=True, cmap='coolwarm', fmt=".2f", square=True)
    plt.title('Correlation Heatmap (Latest Date)')
    plt.tight_layout()
    plt.show()

    sns.set(style='whitegrid')

plt.rcParams['figure.figsize'] = (12, 6)

# --- Step 1: Ensure vaccination-related columns exist and are clean ---

vacc_cols = ['total_vaccinations', 'people_vaccinated', 'population']

for col in vacc_cols:

    if col not in df.columns:

        print(f"Warning: Column '{col}' not found in the dataset.")

    else:

        df[col] = pd.to_numeric(df[col], errors='coerce')

# Forward-fill population per country assuming it is relatively stable

if 'population' in df.columns:

    df.sort_values(['country', 'date'], inplace=True)

    df['population'] = df.groupby('country')['population'].ffill().bfill()

else:

    print("Warning: 'population' column missing. Some visualizations may be limited.")

# Fill missing vaccination data with 0 to ensure continuity

for col in ['total_vaccinations', 'people_vaccinated']:

    if col in df.columns:

        df[col] = df[col].fillna(0)

# Convert 'date' column to datetime

df['date'] = pd.to_datetime(df['date'], errors='coerce')

# --- Step 2: Plot cumulative vaccinations over time for selected countries ---

selected_countries = ['USA', 'India', 'Brazil']  # Customize as needed

plt.figure()

for country in selected_countries:

    country_data = df[df['country'] == country].sort_values('date')

    if not country_data.empty and 'total_vaccinations' in country_data.columns:

        plt.plot(country_data['date'], country_data['total_vaccinations'], marker='o', label=country)

    else:

        print(f"No cumulative vaccination data available for {country}.")

plt.title('Cumulative COVID-19 Vaccinations Over Time')

plt.xlabel('Date')

plt.ylabel('Total Vaccinations Administered')

plt.legend()

plt.xticks(rotation=45)

plt.tight_layout()

plt.show()
